In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import json
import time

res = requests.get("https://www.fantasypros.com/nfl/rankings/rookies.php")
soup = BS(res.content, "html.parser")

In [2]:
# var adpData
scripts = soup.find_all("script")
# script

In [3]:
script = [script for script in scripts if "adpData" in str(script)][0]
# script

In [4]:
script_lines = str(script).split("\n")
# script_lines

In [5]:
ecrData = [line for line in script_lines if "ecrData" in line][0]
# ecrData

In [6]:
player_data = json.loads(ecrData.replace("    var ecrData = ","").replace(";",""))

player_data = player_data['players']

# player_data

In [7]:
data_list = []

for player in player_data:
    player_name = player['player_name']
    player_id = player['player_id']
    position = player['player_positions']
    team = player['player_team_id']
    age = player['player_age']
   
    
    pfjsonp = "680"
    api_key = "2efb16f8f36d0897c3e188b67e3292d5"
    url = f"https://partners.fantasypros.com/api/v1/nfl-player.php?api_key={api_key}&callback=pfjsonp_{pfjsonp}&player={player_id}&week=current&scoring=STD"
    res = requests.get(url).text
    player_stats = json.loads(res.replace(f"pfjsonp_{pfjsonp}(","").replace(");",""))
    data = player_stats['stats'][0]['stats']
    data['Player'] = player_name
    data['position'] = position
    data['player_team_id'] = team
    data['player_age'] = age
    
    
    data_list.append(data)

In [8]:
stats_df = pd.DataFrame(data_list)
stats_df

,POINTS,ATT,YDS,TDS,REC,Player,position,player_team_id,player_age,CMP,CMP %,INTS
0,176.8,219.6,991.7,6.7,39.3,Breece Hall,RB,NYJ,21,NaN,NaN,NaN
1,113.5,NaN,863.7,4.7,65.5,Drake London,WR,ATL,21,NaN,NaN,NaN
2,105.6,164.8,643.8,4.6,17.6,Ken Walker III,RB,SEA,21,NaN,NaN,NaN
3,116.2,NaN,834.8,5,58.7,Treylon Burks,WR,TEN,22,NaN,NaN,NaN
4,70.3,NaN,526,3.1,38.3,Jameson Williams,WR,DET,21,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
89,0,0,0,0,0,Master Teague,RB,FA,22,NaN,NaN,NaN
90,0,NaN,0,0,0,Ty Fryfogle,WR,FA,23,NaN,NaN,NaN
91,0,0,0,0,NaN,Jack Coan,QB,IND,23,0,0,0
92,0,NaN,0,0,0,Reggie Roberson,WR,FA,None,NaN,NaN,NaN


In [9]:
stats_df.to_csv('raw_rookie_stats.csv')

In [10]:
index_df = stats_df.set_index('Player').reset_index()
index_df

,Player,POINTS,ATT,YDS,TDS,REC,position,player_team_id,player_age,CMP,CMP %,INTS
0,Breece Hall,176.8,219.6,991.7,6.7,39.3,RB,NYJ,21,NaN,NaN,NaN
1,Drake London,113.5,NaN,863.7,4.7,65.5,WR,ATL,21,NaN,NaN,NaN
2,Ken Walker III,105.6,164.8,643.8,4.6,17.6,RB,SEA,21,NaN,NaN,NaN
3,Treylon Burks,116.2,NaN,834.8,5,58.7,WR,TEN,22,NaN,NaN,NaN
4,Jameson Williams,70.3,NaN,526,3.1,38.3,WR,DET,21,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
89,Master Teague,0,0,0,0,0,RB,FA,22,NaN,NaN,NaN
90,Ty Fryfogle,0,NaN,0,0,0,WR,FA,23,NaN,NaN,NaN
91,Jack Coan,0,0,0,0,NaN,QB,IND,23,0,0,0
92,Reggie Roberson,0,NaN,0,0,0,WR,FA,None,NaN,NaN,NaN


In [11]:
drop_df = index_df.drop(columns=['CMP','CMP %','INTS'])
drop_df

,Player,POINTS,ATT,YDS,TDS,REC,position,player_team_id,player_age
0,Breece Hall,176.8,219.6,991.7,6.7,39.3,RB,NYJ,21
1,Drake London,113.5,NaN,863.7,4.7,65.5,WR,ATL,21
2,Ken Walker III,105.6,164.8,643.8,4.6,17.6,RB,SEA,21
3,Treylon Burks,116.2,NaN,834.8,5,58.7,WR,TEN,22
4,Jameson Williams,70.3,NaN,526,3.1,38.3,WR,DET,21
...,...,...,...,...,...,...,...,...,...
89,Master Teague,0,0,0,0,0,RB,FA,22
90,Ty Fryfogle,0,NaN,0,0,0,WR,FA,23
91,Jack Coan,0,0,0,0,NaN,QB,IND,23
92,Reggie Roberson,0,NaN,0,0,0,WR,FA,None


In [12]:
# drop_df.loc[drop_df['position'] == 'WR']

In [13]:
# drop_df.loc[drop_df['position'] == 'RB']

In [14]:
reorder_df = drop_df.fillna(0)
# reorder_df

In [15]:
reorder_df = reorder_df[['Player','player_team_id', 'REC', 'YDS', 'TDS','POINTS','position']]
reorder_df

,Player,player_team_id,REC,YDS,TDS,POINTS,position
0,Breece Hall,NYJ,39.3,991.7,6.7,176.8,RB
1,Drake London,ATL,65.5,863.7,4.7,113.5,WR
2,Ken Walker III,SEA,17.6,643.8,4.6,105.6,RB
3,Treylon Burks,TEN,58.7,834.8,5,116.2,WR
4,Jameson Williams,DET,38.3,526,3.1,70.3,WR
...,...,...,...,...,...,...,...
89,Master Teague,FA,0,0,0,0,RB
90,Ty Fryfogle,FA,0,0,0,0,WR
91,Jack Coan,IND,0,0,0,0,QB
92,Reggie Roberson,FA,0,0,0,0,WR


In [16]:
rename_df = reorder_df.rename(columns={'player_team_id':'2021 Tm','REC':'2021 Rec', 'YDS':'2021 RushingYds','TDS':'2021 ReceivingTD', 'POINTS':'2021 Fantasy Points','position':'Pos'})

rename_df

,Player,2021 Tm,2021 Rec,2021 RushingYds,2021 ReceivingTD,2021 Fantasy Points,Pos
0,Breece Hall,NYJ,39.3,991.7,6.7,176.8,RB
1,Drake London,ATL,65.5,863.7,4.7,113.5,WR
2,Ken Walker III,SEA,17.6,643.8,4.6,105.6,RB
3,Treylon Burks,TEN,58.7,834.8,5,116.2,WR
4,Jameson Williams,DET,38.3,526,3.1,70.3,WR
...,...,...,...,...,...,...,...
89,Master Teague,FA,0,0,0,0,RB
90,Ty Fryfogle,FA,0,0,0,0,WR
91,Jack Coan,IND,0,0,0,0,QB
92,Reggie Roberson,FA,0,0,0,0,WR


In [17]:
adp =pd.read_csv('../Ronnie/data/2022_ADP_Rankings.csv') 

adp

,Rank,Player,Team,Bye,POS,Yahoo,Fantrax,FFC,Sleeper,AVG
0,1,Jonathan Taylor,IND,14.0,RB1,1.0,1.0,1.0,1.0,1.0
1,2,Austin Ekeler,LAC,8.0,RB2,3.0,2.0,3.0,4.0,3.0
2,3,Cooper Kupp,LAR,7.0,WR1,2.0,3.0,4.0,5.0,3.5
3,4,Derrick Henry,TEN,6.0,RB3,5.0,5.0,2.0,3.0,3.8
4,5,Christian McCaffrey,CAR,13.0,RB4,4.0,4.0,6.0,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...
399,400,Dee Eskridge,SEA,11.0,WR141,NaN,400.0,NaN,NaN,400.0
400,401,Donald Parham Jr.,LAC,8.0,TE48,NaN,401.0,NaN,NaN,401.0
401,402,Jordan Love,GB,14.0,QB43,NaN,402.0,NaN,NaN,402.0
402,403,Matt Corral,CAR,13.0,QB44,NaN,403.0,NaN,NaN,403.0


In [18]:
merge_df = pd.merge(rename_df,adp, on = 'Player', how = 'left')
merge_df

,Player,2021 Tm,2021 Rec,2021 RushingYds,2021 ReceivingTD,2021 Fantasy Points,Pos,Rank,Team,Bye,POS,Yahoo,Fantrax,FFC,Sleeper,AVG
0,Breece Hall,NYJ,39.3,991.7,6.7,176.8,RB,43.0,NYJ,10.0,RB20,39.0,39.0,57.0,37.0,43.0
1,Drake London,ATL,65.5,863.7,4.7,113.5,WR,104.0,ATL,14.0,WR41,123.0,99.0,145.0,76.0,110.8
2,Ken Walker III,SEA,17.6,643.8,4.6,105.6,RB,89.0,SEA,11.0,RB34,103.0,88.0,87.0,73.0,87.8
3,Treylon Burks,TEN,58.7,834.8,5,116.2,WR,119.0,TEN,6.0,WR48,174.0,94.0,141.0,83.0,123.0
4,Jameson Williams,DET,38.3,526,3.1,70.3,WR,163.0,DET,6.0,WR63,199.0,142.0,191.0,136.0,167.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Master Teague,FA,0,0,0,0,RB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,Ty Fryfogle,FA,0,0,0,0,WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,Jack Coan,IND,0,0,0,0,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,Reggie Roberson,FA,0,0,0,0,WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
dmerge = merge_df.drop(columns=['Rank','Team','Bye','POS','Yahoo','Fantrax','FFC','Sleeper'])
dmerge

,Player,2021 Tm,2021 Rec,2021 RushingYds,2021 ReceivingTD,2021 Fantasy Points,Pos,AVG
0,Breece Hall,NYJ,39.3,991.7,6.7,176.8,RB,43.0
1,Drake London,ATL,65.5,863.7,4.7,113.5,WR,110.8
2,Ken Walker III,SEA,17.6,643.8,4.6,105.6,RB,87.8
3,Treylon Burks,TEN,58.7,834.8,5,116.2,WR,123.0
4,Jameson Williams,DET,38.3,526,3.1,70.3,WR,167.0
...,...,...,...,...,...,...,...,...
89,Master Teague,FA,0,0,0,0,RB,NaN
90,Ty Fryfogle,FA,0,0,0,0,WR,NaN
91,Jack Coan,IND,0,0,0,0,QB,NaN
92,Reggie Roberson,FA,0,0,0,0,WR,NaN


In [20]:
cleaned_df = dmerge.loc[(rename_df['2021 Fantasy Points'].astype(float) > 0)]
cleaned_df

,Player,2021 Tm,2021 Rec,2021 RushingYds,2021 ReceivingTD,2021 Fantasy Points,Pos,AVG
0,Breece Hall,NYJ,39.3,991.7,6.7,176.8,RB,43.0
1,Drake London,ATL,65.5,863.7,4.7,113.5,WR,110.8
2,Ken Walker III,SEA,17.6,643.8,4.6,105.6,RB,87.8
3,Treylon Burks,TEN,58.7,834.8,5,116.2,WR,123.0
4,Jameson Williams,DET,38.3,526,3.1,70.3,WR,167.0
...,...,...,...,...,...,...,...,...
72,Jalen Nailor,MIN,3.4,39.9,0.3,5.8,WR,350.0
73,Samori Toure,GB,6.8,80.6,0.7,12.4,WR,NaN
74,Michael Woods II,FA,3.9,60.5,0.3,8.1,WR,NaN
76,Tyler Goodson,GB,9.1,198.1,1.9,40.2,RB,NaN


In [21]:
QB_df = cleaned_df.loc[(cleaned_df['Pos']== 'QB')].drop(columns='2021 Rec').rename(columns={'2021 ReceivingTD':'Touchdowns21'})
QB_df

,Player,2021 Tm,2021 RushingYds,Touchdowns21,2021 Fantasy Points,Pos,AVG
19,Kenny Pickett,PIT,1559,9.2,102.4,QB,265.5
23,Malik Willis,TEN,188.7,1,12.1,QB,316.0
27,Desmond Ridder,ATL,1189.8,6.2,80.5,QB,287.0
40,Sam Howell,WAS,109.2,0.8,8.1,QB,NaN


In [22]:
QB_df.to_csv('Rookies_QB.csv')

In [23]:
Other_df = cleaned_df.loc[(cleaned_df['Pos']!= 'QB')]
Other_df

,Player,2021 Tm,2021 Rec,2021 RushingYds,2021 ReceivingTD,2021 Fantasy Points,Pos,AVG
0,Breece Hall,NYJ,39.3,991.7,6.7,176.8,RB,43.0
1,Drake London,ATL,65.5,863.7,4.7,113.5,WR,110.8
2,Ken Walker III,SEA,17.6,643.8,4.6,105.6,RB,87.8
3,Treylon Burks,TEN,58.7,834.8,5,116.2,WR,123.0
4,Jameson Williams,DET,38.3,526,3.1,70.3,WR,167.0
...,...,...,...,...,...,...,...,...
72,Jalen Nailor,MIN,3.4,39.9,0.3,5.8,WR,350.0
73,Samori Toure,GB,6.8,80.6,0.7,12.4,WR,NaN
74,Michael Woods II,FA,3.9,60.5,0.3,8.1,WR,NaN
76,Tyler Goodson,GB,9.1,198.1,1.9,40.2,RB,NaN


In [24]:
Other_df.to_csv('Rookies_RB_WR_TE.csv')